In [3]:
import cv2
import numpy as np

img1 = cv2.imread('image1.jpg')
img2 = cv2.imread('image2.jpg')
img3 = cv2.imread('image3.jpg')

# Create a rotation matrix that will rotate the images so that the features in the images are aligned horizontally
M1 = cv2.getRotationMatrix2D((img1.shape[1]/2, img1.shape[0]/2), 90, 1)
M2 = cv2.getRotationMatrix2D((img2.shape[1]/2, img2.shape[0]/2), 180, 1)
M3 = cv2.getRotationMatrix2D((img3.shape[1]/2, img3.shape[0]/2), 270, 1)

img1 = cv2.warpAffine(img1, M1, (img1.shape[1], img1.shape[0]))
img2 = cv2.warpAffine(img2, M2, (img2.shape[1], img2.shape[0]))
img3 = cv2.warpAffine(img3, M3, (img3.shape[1], img3.shape[0]))

# Compute the perspective transformation that will map the images to a cylindrical surface
h, w = img1.shape[:2]
dst = cv2.getPerspectiveTransform(np.float32([[0, 0], [w, 0], [w, h], [0, h]]), np.float32([[0, 0], [w, 0], [w, h], [0, h]]))

# Warp the images to the cylindrical surface
img1 = cv2.warpPerspective(img1, dst, (w, h))
img2 = cv2.warpPerspective(img2, dst, (w, h))
img3 = cv2.warpPerspective(img3, dst, (w, h))

# Blend the overlapping areas of the images
panorama = cv2.addWeighted(img1, 0.5, img2, 0.5, 0)
panorama = cv2.addWeighted(panorama, 0.5, img3, 0.5, 0)

cv2.imshow('panorama', panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
import cv2
import glob
import numpy as np

# Create an empty array to hold the transformed images
transformed_images = []

# Loop through the images
for image in ['image1.jpg', 'image2.jpg', 'image3.jpg']:
    # Read the image
    img = cv2.imread(image)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect the features in the image using ORB
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(gray, None)

    # Create a mask to draw the features on
    mask = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    # Loop through the other images
    for other_image in ['image1.jpg', 'image2.jpg', 'image3.jpg']:
        if other_image == image:
            continue

        # Read the other image
        img2 = cv2.imread(other_image)

        # Convert the other image to grayscale
        gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

        # Detect the features in the other image using ORB
        kp2, des2 = orb.detectAndCompute(gray2, None)

        # Use brute force to find the matches between the two images
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = bf.match(des1, des2)

        # Sort the matches by distance
        matches = sorted(matches, key=lambda x: x.distance)

        # Take the top N matches
        N = 50
        matches = matches[:N]

        # Get the coordinates of the matches
        pts1 = np.array([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        pts2 = np.array([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

        # Find the transformation matrix using the matches
        H, _ = cv2.findHomography(pts1, pts2, cv2.RANSAC)

        # Warp the image using the transformation matrix
        img_warped = cv2.warpPerspective(img, H, (img.shape[1], img.shape[0]))

        # Add the transformed image to the list of transformed images
        transformed_images.append(img_warped)

# Combine the transformed images into a single panorama
panorama = cv2.hconcat(transformed_images)

# Save the resulting panorama
cv2.imwrite("panorama.jpg", panorama)

True